In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')
import numpy as np
import logging
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time

libc = CDLL(find_library('c'))

cstdout = c_void_p.in_dll(libc, 'stdout')

In [ ]:
class layers:

    def __init__(self):
        self.layertypes = []

        self.weights = []

        self.biases = []

        self.numlayer = 0

        self.ffn_counter = 0


def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")

    if text[-1] != ']':
        raise Exception("expected ']'")

    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])

    # return v.reshape((v.size,1))

    return v


def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")

    if text[-1] != ']':
        raise Exception("expected ']'")

    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])

    return v.reshape((v.size, 1))

    # return v


def balanced_split(text):
    i = 0

    bal = 0

    start = 0

    result = []

    while i < len(text):

        if text[i] == '[':

            bal += 1

        elif text[i] == ']':

            bal -= 1

        elif text[i] == ',' and bal == 0:

            result.append(text[start:i])

            start = i + 1

        i += 1

    if start < i:
        result.append(text[start:i])

    return result


def parse_matrix(text):
    i = 0

    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")

    if text[-1] != ']':
        raise Exception("expected ']'")

    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])


def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]

    i = 0

    res = layers()

    while i < len(lines):

        if lines[i] in ['ReLU', 'Affine']:

            W = parse_matrix(lines[i + 1])

            b = parse_bias(lines[i + 2])

            res.layertypes.append(lines[i])

            res.weights.append(W)

            res.biases.append(b)

            res.numlayer += 1

            i += 3

        else:

            raise Exception('parse error: ' + lines[i])

    return res


def parse_spec(text):
    text = text.replace("[", "")

    text = text.replace("]", "")

    with open('dummy', 'w') as my_file:
        my_file.write(text)

    data = np.genfromtxt('dummy', delimiter=',', dtype=np.double)

    low = np.copy(data[:, 0])

    high = np.copy(data[:, 1])

    return low, high


def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]

    num_pixels = len(image)

    LB_N0 = image - epsilon

    UB_N0 = image + epsilon

    for i in range(num_pixels):

        if (LB_N0[i] < 0):
            LB_N0[i] = 0

        if (UB_N0[i] > 1):
            UB_N0[i] = 1

    return LB_N0, UB_N0



def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)

    cst = pointer(linexpr0.contents.cst)

    elina_scalar_set_double(cst.contents.val.scalar, bias)

    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0, i, weights[i])

    return linexpr0